In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Modules

In [2]:
import pandas as pd
import librosa
import numpy as np
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import ast

In [ ]:
# Checking weather path is correct or not
path = '/content/drive/MyDrive/Dysarthia/audio/F02/fsr/F02_B1_C1_M2_FSR.wav'
ipd.Audio(path)

## Loading the Data

In [3]:
training_filepath = '/content/drive/MyDrive/Dysarthia/csv_Files/drive_final_fsr_uastrain.csv'
df_train = pd.read_csv(training_filepath)

## Analysing the Data

In [ ]:
df_train.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration
0,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M2.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376
1,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M3.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376
2,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376
3,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376
4,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.408


In [ ]:
df_train.shape

(29923, 7)

In [ ]:
df_train.columns

Index(['speaker', 'word_type', 'path', 'srpath', 'label', 'duration',
       'trimsoundduration'],
      dtype='object')

In [ ]:
type(df_train.srpath[0])

str

In [ ]:
df_train.srpath[0]

'/content/drive/MyDrive/Dysarthia/audio/F02/fsr/F02_B1_C10_M2_FSR.wav'

In [ ]:
sample_path = df_train.srpath[0]
ipd.Audio(sample_path)

## Loading the Audio Data as a Column

In [4]:
# Parameter for the Output
input_dim = 40
num_classes = 4
win_length = 400
n_fft = 512
hop_length = 80
n_mels = 40
n_mfcc = 40
spec_len=400
sr = 16000

In [5]:
#Loading Audio files with the help of file path

def load_wav(audio_filepath, sr, input_length_time=1):
    data,fs  = librosa.load(audio_filepath, sr=sr)
    input_length = input_length_time * sr
    #len_file = len(audio_data)
    #if len_file <int(min_dur_sec*sr):
    #    dummy=np.zeros((1,int(min_dur_sec*sr)-len_file))
    #    extened_wav = np.concatenate((audio_data,dummy[0]))
    #else:

    #    extened_wav = audio_data
    if len(data) > input_length:
        max_offset = len(data) - input_length
        offset = np.random.randint(max_offset)
        data = data[offset:(input_length + offset)]
    else:
        if input_length > len(data):
            max_offset = input_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, input_length - len(data) - offset), "constant")
    return data

In [7]:
# Apply the function using a lambda to pass the sr argument
df_train['audio_data'] = df_train['srpath'].apply(lambda x: load_wav(x, sr=16000))

In [8]:
df_train.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data
0,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M2.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[0.017944336, 0.033813477, 0.048675537, 0.0561..."
1,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M3.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[0.016662598, -0.07095337, -0.23947144, -0.268..."
2,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[-0.04788208, 0.021759033, 0.11538696, 0.11843..."
3,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[-0.07879639, -0.0357666, -0.07507324, -0.1955..."
4,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.408,"[0.08166504, 0.06555176, 0.037506104, 0.009338..."


In [9]:
len(df_train.audio_data[0])

16000

In [10]:
type(df_train.audio_data[0])

numpy.ndarray

In [ ]:
# audio_data_column = df_train['audio_data'].values

In [ ]:
# audio_data_npy_filePath = '/content/drive/MyDrive/Dysarthia'
# np.save(audio_data_npy_filePath, audio_data_column)

## Calculating Melspectrogram as a Column

In [ ]:
def lin_mel_from_wav(wav, hop_length, win_length, n_mels):
    linear = librosa.feature.melspectrogram(y = wav, n_mels=n_mels, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T

In [ ]:
# Apply the function using a lambda to pass the sr argument
df_train['Melspectrogram'] = df_train['audio_data'].apply(lambda x: lin_mel_from_wav(wav = x, n_mels = input_dim, win_length = win_length,
                                                                                     hop_length = 256))

In [ ]:
df_train.head()

In [ ]:
df_train.Melspectrogram[0]

In [ ]:
print(df_train.Melspectrogram.dtype)

## Visualisation of Melspectrogram

In [ ]:
log_mel_scale = librosa.power_to_db(df_train.Melspectrogram[0])

In [ ]:
def plot_mel_spectrogram(y, sr):
    plt.figure(figsize = (25,10))
    librosa.display.specshow(y, x_axis = "time", y_axis = "mel", sr = sr)
    plt.colorbar(format = "%+2.f")

In [ ]:
plot_mel_spectrogram(df_train.Melspectrogram[0], sr = 16000)

## Calculating STFT

In [ ]:
def lin_spectogram_from_wav(wav, hop_length, win_length, n_fft=512):
    linear = librosa.stft(y = wav, n_fft=n_fft, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T

In [ ]:
sample = df_train.audio_data[0]
x = lin_spectogram_from_wav(wav = sample, hop_length = 256, win_length = win_length, n_fft = n_fft)
x.shape

In [ ]:
df_train['STFT'] = df_train['audio_data'].apply(lambda x: lin_spectogram_from_wav(wav = x, hop_length = 256,win_length = win_length, n_fft = n_fft))

In [ ]:
df_train.head()

In [ ]:
# @title label

from matplotlib import pyplot as plt
import seaborn as sns
df_train.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_train.STFT[0]

In [ ]:
df_train.STFT[0].shape

## Visualising STFT

In [ ]:
STFT_sample = df_train.STFT[0]
Abs_STFT_sample = np.abs(STFT_sample)
log_Abs_STFT_sample = librosa.power_to_db(Abs_STFT_sample)

In [ ]:
def plot_stft_spectrogram(y, sr, hop_length, y_axis = "log"):
    plt.figure(figsize = (25,10))
    librosa.display.specshow(y, sr = sr, hop_length = hop_length, x_axis = "time", y_axis = y_axis)
    plt.colorbar(format = "%+2.f")

In [ ]:
plot_stft_spectrogram(log_Abs_STFT_sample, sr = sr, hop_length = hop_length)

## Calculating MFCC

In [11]:
def lin_mfcc_from_wav(wav, hop_length, win_length, n_mfcc, n_fft=512):
    linear = librosa.feature.mfcc(y=wav, n_fft=n_fft, win_length=win_length, hop_length=hop_length, n_mfcc=n_mfcc, sr=16000) # mfcc
    return linear.T

In [12]:
df_train['MFCC'] = df_train['audio_data'].apply(lambda x: lin_mfcc_from_wav(wav = x, hop_length = hop_length, win_length = win_length, n_fft = n_fft, n_mfcc = n_mfcc))

In [13]:
df_train.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data,MFCC
0,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M2.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[0.017944336, 0.033813477, 0.048675537, 0.0561...","[[-344.99918, 95.819, -20.402824, 44.37455, 4...."
1,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M3.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[0.016662598, -0.07095337, -0.23947144, -0.268...","[[-206.55153, 68.87056, -56.56128, 11.210167, ..."
2,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[-0.04788208, 0.021759033, 0.11538696, 0.11843...","[[-237.27429, 62.78328, -62.513664, 6.073738, ..."
3,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.376,"[-0.07879639, -0.0357666, -0.07507324, -0.1955...","[[-221.45058, 55.18961, -20.683586, 2.745111, ..."
4,F02,CMD,C:\uaspeech\UASpeech\audio\F02\F02_B1_C10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,4.383687,1.408,"[0.08166504, 0.06555176, 0.037506104, 0.009338...","[[-297.06866, 93.51008, -1.5811696, 32.051567,..."


In [14]:
df_train.MFCC[0]

array([[-3.44999176e+02,  9.58190002e+01, -2.04028244e+01, ...,
         5.95782995e-02,  1.60621309e+00, -6.14013290e+00],
       [-3.36970459e+02,  7.41019287e+01, -2.29062309e+01, ...,
        -7.63865566e+00, -2.17319012e+00, -4.69331694e+00],
       [-3.45354156e+02,  5.82055664e+01, -2.88655014e+01, ...,
        -1.63999043e+01, -5.99797583e+00, -6.49277258e+00],
       ...,
       [-3.00008392e+02,  9.04409103e+01,  3.22525330e+01, ...,
        -6.32287407e+00, -2.06186819e+00,  6.06908798e+00],
       [-2.35990799e+02,  1.01228714e+02,  9.30224991e+00, ...,
         8.38491559e-01,  1.69357276e+00,  1.34378123e+00],
       [-2.23667969e+02,  1.03821350e+02,  2.74163532e+00, ...,
        -4.96638805e-01,  1.13748193e+00,  1.38480306e-01]], dtype=float32)

In [15]:
df_train.MFCC[55].shape

(201, 40)

In [16]:
type(df_train.MFCC[0])

numpy.ndarray

In [ ]:
# mfcc_coulmn = df_train['MFCC'].values

# mfcc_npy_filePath = '/content/drive/MyDrive/Dysarthia/mfcc.npy'
# np.save(mfcc_npy_filePath, mfcc_coulmn)

In [ ]:
# df_train.to_csv('/content/drive/MyDrive/Dysarthia/traning_df_with_mfcc.csv', index=False)

In [17]:
df_train['label'].unique()

array(['LO', 'VL', 'MI', 'HI'], dtype=object)

In [18]:
def extract_mfcc_for_VL(df):
    # Group the DataFrame by the 'Label' column
    grouped = df.groupby('label')

    # Extract the mfcc values for VL
    mfcc_VL = grouped.get_group('VL')['MFCC'].values

    return mfcc_VL

def extract_mfcc_for_LO(df):
    # Group the DataFrame by the 'Label' column
    grouped = df.groupby('label')

    # Extract the mfcc values for VL
    mfcc_LO = grouped.get_group('LO')['MFCC'].values

    return mfcc_LO

def extract_mfcc_for_MI(df):
    # Group the DataFrame by the 'Label' column
    grouped = df.groupby('label')

    # Extract the mfcc values for MI
    mfcc_MI = grouped.get_group('MI')['MFCC'].values

    return mfcc_MI

def extract_mfcc_for_HI(df):
    # Group the DataFrame by the 'Label' column
    grouped = df.groupby('label')

    # Extract the mfcc values for VL
    mfcc_HI = grouped.get_group('HI')['MFCC'].values

    return mfcc_HI

In [ ]:
train_VL = extract_mfcc_for_VL(df_train)
train_L = extract_mfcc_for_LO(df_train)
train_MI = extract_mfcc_for_MI(df_train)
train_HI = extract_mfcc_for_HI(df_train)

In [ ]:
df_train['label'].value_counts()

,count
label,
LO,7818
HI,7818
MI,7714
VL,6573


In [ ]:
train_VL.shape

(6573,)

In [ ]:
train_L.shape

(7818,)

In [ ]:
train_HI.shape

(7818,)

In [ ]:
train_MI.shape

(7714,)

In [ ]:
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_uastrain/train_VL.npy', train_VL)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_uastrain/train_LO.npy', train_L)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_uastrain/train_MI.npy', train_MI)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_uastrain/train_HI.npy', train_HI)

In [19]:
df_train.to_csv('/content/drive/MyDrive/Dysarthia/Final_features/fsr_uastrain/CSV_train_df_with_mfcc.csv')

## fsr_sdtd

In [20]:
df_sdtd = pd.read_csv('/content/drive/MyDrive/Dysarthia/csv_Files/drive_final_fsr_sdtd.csv')

In [21]:
df_sdtd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.792
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.120
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW11_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.749625,2.880


In [22]:
# Apply the function using a lambda to pass the sr argument
df_sdtd['audio_data'] = df_sdtd['srpath'].apply(lambda x: load_wav(x, sr=16000))

In [23]:
df_sdtd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.792,"[0.5480652, 0.49014282, 0.42184448, 0.3008728,..."
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.120,"[0.06866455, -0.025543213, 0.0708313, -0.05093..."
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW11_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.749625,2.880,"[0.008636475, 0.008422852, 0.009185791, 0.0079..."


In [24]:
df_sdtd['MFCC'] = df_sdtd['audio_data'].apply(lambda x: lin_mfcc_from_wav(wav = x, hop_length = hop_length, win_length = win_length, n_fft = n_fft, n_mfcc = n_mfcc))

In [25]:
df_sdtd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data,MFCC
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-724.14795, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,3.143500,0.960,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-737.854, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.792,"[0.5480652, 0.49014282, 0.42184448, 0.3008728,...","[[-148.6449, 105.963104, 8.452635, 1.162946, -..."
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.505375,1.120,"[0.06866455, -0.025543213, 0.0708313, -0.05093...","[[-327.73404, -85.4147, -1.5837011, 8.9452305,..."
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B2_UW11_M4.wav,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,5.749625,2.880,"[0.008636475, 0.008422852, 0.009185791, 0.0079...","[[-507.52545, 106.642395, 49.171158, 22.008354..."


In [26]:
df_sdtd['label'].value_counts()

,count
label,
LO,400
MI,400
HI,400
VL,300


In [27]:
sdtd_VL = extract_mfcc_for_VL(df_sdtd)
sdtd_LO = extract_mfcc_for_LO(df_sdtd)
sdtd_MI = extract_mfcc_for_MI(df_sdtd)
sdtd_HI = extract_mfcc_for_HI(df_sdtd)

In [30]:
sdtd_VL.shape

(300,)

In [ ]:
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdtd/stdt_VL.npy', sdtd_VL)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdtd/stdt_LO.npy', sdtd_LO)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdtd/stdt_MI.npy', sdtd_MI)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdtd/stdt_HI.npy', sdtd_HI)

In [31]:
df_sdtd.to_csv('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdtd/sdtd_df_with_mfcc.csv')

## fsr_sdti

In [32]:
df_sdti = pd.read_csv('/content/drive/MyDrive/Dysarthia/csv_Files/drive_final_fsr_sdti.csv')

In [33]:
df_sdti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M2...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.72800
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M3...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.85525
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.47200


In [35]:
# Apply the function using a lambda to pass the sr argument
df_sdti['audio_data'] = df_sdti['srpath'].apply(lambda x: load_wav(x, sr=16000))

In [36]:
df_sdti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M2...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.72800,"[-0.006134033, -0.004425049, -0.0033874512, -0..."
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M3...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325,"[-0.08300781, -0.0035705566, 0.012573242, -0.0..."
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.85525,"[0.11929321, 0.17260742, 0.21704102, 0.2611389..."
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325,"[-0.00015258789, 0.0012512207, 0.0046691895, 0..."
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.47200,"[0.09863281, 0.090789795, 0.08013916, 0.069427..."


In [37]:
df_sdti['MFCC'] = df_sdti['audio_data'].apply(lambda x: lin_mfcc_from_wav(wav = x, hop_length = hop_length, win_length = win_length, n_fft = n_fft, n_mfcc = n_mfcc))

In [38]:
df_sdti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data,MFCC
0,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M2...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.72800,"[-0.006134033, -0.004425049, -0.0033874512, -0...","[[-518.02673, 132.46812, 69.299484, 19.04694, ..."
1,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M3...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325,"[-0.08300781, -0.0035705566, 0.012573242, -0.0...","[[-195.6157, 77.703285, -16.948242, 15.282587,..."
2,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M4...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.85525,"[0.11929321, 0.17260742, 0.21704102, 0.2611389...","[[-281.33817, 151.13397, 37.595734, 14.274458,..."
3,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.82325,"[-0.00015258789, 0.0012512207, 0.0046691895, 0...","[[-495.86407, 152.76208, 70.75183, -5.264412, ..."
4,F02,UW,C:\uaspeech\UASpeech\audio\F02\F02_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/F02/fsr...,LO,6.65525,1.47200,"[0.09863281, 0.090789795, 0.08013916, 0.069427...","[[-324.1987, 103.67798, 16.811356, 8.345732, -..."


In [39]:
df_sdti['label'].value_counts()

,count
label,
LO,1400
MI,1400
HI,1400
VL,1200


In [40]:
sdti_VL = extract_mfcc_for_VL(df_sdti)
sdti_LO = extract_mfcc_for_LO(df_sdti)
sdti_MI = extract_mfcc_for_MI(df_sdti)
sdti_HI = extract_mfcc_for_HI(df_sdti)

In [41]:
sdti_VL.shape

(1200,)

In [ ]:
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdti/sdti_VL.npy', sdti_VL)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdti/sdti_LO.npy', sdti_LO)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdti/sdti_MI.npy', sdti_MI)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdti/sdti_HI.npy', sdti_HI)

In [42]:
df_sdtd.to_csv('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sdti/sdti_df_with_mfcc.csv')

## fsr_sitd

In [43]:
df_sitd = pd.read_csv('/content/drive/MyDrive/Dysarthia/csv_Files/drive_final_fsr_sitd.csv')

In [44]:
df_sitd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,4.490250,0.928
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,14.521750,1.760
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW11_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.527000,2.240
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW12_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,13.110687,1.920
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW13_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,7.890125,2.400


In [46]:
# Apply the function using a lambda to pass the sr argument
df_sitd['audio_data'] = df_sitd['srpath'].apply(lambda x: load_wav(x, sr=16000))

In [47]:
df_sitd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,4.490250,0.928,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,14.521750,1.760,"[0.35076904, 0.31463623, 0.32711792, 0.3746338..."
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW11_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.527000,2.240,"[-0.060791016, -0.06774902, -0.06808472, -0.06..."
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW12_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,13.110687,1.920,"[-0.05709839, -0.04058838, -0.008972168, 0.017..."
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW13_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,7.890125,2.400,"[0.021606445, 0.02142334, 0.019805908, 0.01684..."


In [48]:
df_sitd['MFCC'] = df_sitd['audio_data'].apply(lambda x: lin_mfcc_from_wav(wav = x, hop_length = hop_length, win_length = win_length, n_fft = n_fft, n_mfcc = n_mfcc))

In [49]:
df_sitd.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data,MFCC
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,4.490250,0.928,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-733.15826, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,14.521750,1.760,"[0.35076904, 0.31463623, 0.32711792, 0.3746338...","[[-190.26733, 97.9005, 5.4032464, 10.971358, 7..."
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW11_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.527000,2.240,"[-0.060791016, -0.06774902, -0.06808472, -0.06...","[[-320.87006, 139.1578, 8.180571, 38.523525, 3..."
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW12_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,13.110687,1.920,"[-0.05709839, -0.04058838, -0.008972168, 0.017...","[[-347.65353, 117.37717, 36.581696, 54.493217,..."
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B1_UW13_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,7.890125,2.400,"[0.021606445, 0.02142334, 0.019805908, 0.01684...","[[-402.6349, 168.53165, 36.46644, 14.480693, 6..."


In [50]:
df_sitd['label'].value_counts()

,count
label,
HI,1000
VL,500
MI,200
LO,200


In [51]:
sitd_VL = extract_mfcc_for_VL(df_sitd)
sitd_LO = extract_mfcc_for_LO(df_sitd)
sitd_MI = extract_mfcc_for_MI(df_sitd)
sitd_HI = extract_mfcc_for_HI(df_sitd)

In [52]:
sitd_VL.shape

(500,)

In [ ]:
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sitd/sitd_VL.npy', sitd_VL)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sitd/sitd_LO.npy', sitd_LO)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sitd/sitd_MI.npy', sitd_MI)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sitd/sitd_HI.npy', sitd_HI)

In [53]:
df_sitd.to_csv('/content/drive/MyDrive/Dysarthia/Final_features/fsr_sitd/sitd_df_with_mfcc.csv')

## fsr_siti

In [54]:
df_siti = pd.read_csv('/content/drive/MyDrive/Dysarthia/csv_Files/drive_final_fsr_siti.csv')

In [55]:
df_siti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.312
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.472
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M7...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.248
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.280
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.184


In [57]:
# Apply the function using a lambda to pass the sr argument
df_siti['audio_data'] = df_siti['srpath'].apply(lambda x: load_wav(x, sr=16000))

In [58]:
df_siti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.312,"[0.046875, 0.055755615, 0.05545044, 0.05569458..."
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.472,"[-0.04385376, -0.045776367, -0.053619385, -0.0..."
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M7...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.248,"[0.043884277, 0.049102783, 0.051483154, 0.0547..."
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.280,"[-0.0067749023, -0.0044555664, -0.0008544922, ..."
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.184,"[-0.0030212402, 0.012939453, 0.009521484, -0.0..."


In [59]:
df_siti['MFCC'] = df_siti['audio_data'].apply(lambda x: lin_mfcc_from_wav(wav = x, hop_length = hop_length, win_length = win_length, n_fft = n_fft, n_mfcc = n_mfcc))

In [65]:
df_siti.head()

,speaker,word_type,path,srpath,label,duration,trimsoundduration,audio_data,MFCC
0,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M5...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.312,"[0.046875, 0.055755615, 0.05545044, 0.05569458...","[[-367.4213, 91.31047, 40.149933, 23.862743, 1..."
1,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M6...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.472,"[-0.04385376, -0.045776367, -0.053619385, -0.0...","[[-374.17166, 98.57881, 9.205066, -9.4741335, ..."
2,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW100_M7...,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,5.386375,1.248,"[0.043884277, 0.049102783, 0.051483154, 0.0547...","[[-365.34726, 102.98967, 33.590187, 22.81999, ..."
3,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M5.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.280,"[-0.0067749023, -0.0044555664, -0.0008544922, ...","[[-511.22913, 66.19127, 17.1787, 48.819294, 26..."
4,M01,UW,C:\uaspeech\UASpeech\audio\M01\M01_B3_UW10_M6.wav,/content/drive/MyDrive/Dysarthia/audio/M01/fsr...,VL,8.784500,1.184,"[-0.0030212402, 0.012939453, 0.009521484, -0.0...","[[-358.98218, 101.346405, -18.607134, 30.95608..."


In [61]:
df_siti['label'].value_counts()

,count
label,
HI,2100
VL,900
MI,600
LO,600


In [62]:
siti_VL = extract_mfcc_for_VL(df_siti)
siti_LO = extract_mfcc_for_LO(df_siti)
siti_MI = extract_mfcc_for_MI(df_siti)
siti_HI = extract_mfcc_for_HI(df_siti)

In [63]:
siti_VL.shape

(900,)

In [ ]:
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_siti/siti_VL.npy', siti_VL)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_siti/siti_LO.npy', siti_LO)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_siti/siti_MI.npy', siti_MI)
np.save('/content/drive/MyDrive/Dysarthia/Final_features/fsr_siti/siti_HI.npy', siti_HI)

In [64]:
df_sitd.to_csv('/content/drive/MyDrive/Dysarthia/Final_features/fsr_siti/siti_df_with_mfcc.csv')